In [2]:
from datasets.datasets import *

In [ ]:
NL = [5]
combined_dataset = get_combined_dataset(NL=NL)

In [3]:
# PLAYER DATASET
#player_dataset = get_player_dataset(NL=NL)

reading file
Loaded 13793 rows.
Normalizing features
['GP', 'G', 'A', 'PTS', 'PS', 'EV', 'PP', 'S']
Loading player data
creating dataset structure
creating player dict


In [4]:
# TEAM DATASET
#team_dataset = get_team_dataset(NL=[1]) # Only need one season (ensures all teams exists in dataset)

Targets
['W%', 'L%', 'S']
Normalizing features
[]
All features
['W%', 'L%', 'GF/G', 'GA/G', 'PIM/G', 'oPIM/G', 'S', 'S%', 'SV%']
Loading player data
creating dataset structure


In [6]:

team_name_dict = {
    'AFM': 'Atlanta Flames',
    'ANA': 'Mighty Ducks of Anaheim/Anaheim Ducks',
    'ARI' : 'Arizona Coyotes',
    'ATL' : 'Atlanta Thrashers',
    'BOS' : 'Boston Bruins',
    'BRK' : 'Brooklyn Americans',
    'BUF' : 'Buffalo Sabres',
    'CAR' : 'Carolina Hurricanes',
    'CBJ' : 'Columbus Blue Jackets',
    'CGS' : 'Bay Area Seals/California Golden Seals',
    'CGY' : 'Calgary Flames',
    'CHI' : 'Chicago Black Hawks/Blackhawks',
    'CLE' : 'Cleveland Barons',
    'CLR' : 'Colorado Rockies',
    'COL' : 'Colorado Avalanche',
    'DAL' : 'Dallas Stars',
    'DCG' : 'Detroit Cougars',
    'DET' : 'Detroit Red Wings',
    'DFL' : 'Detroit Falcons',
    'EDM' : 'Edmonton Oilers',
    'FLA' : 'Florida Panthers',
    'HAM' : 'Hamilton Tigers',
    'HFD' : 'Hartford Whalers',
    'KCS' : 'Kansas City Scouts',
    'LAK' : 'Los Angeles Kings',
    'MIN' : 'Minnesota Wild',
    'MMR' : 'Montreal Maroons',
    'MNS' : 'Minnesota North Stars',
    'MTL' : 'Montreal Canadiens',
    'MWN' : 'Montreal Wanderers',
    'NJD' : 'New Jersey Devils',
    'NSH' : 'Nashville Predators',
    'NYA' : 'New York Americans',
    'NYI' : 'New York Islanders',
    'NYR' : 'New York Rangers',
    'OAK' : 'California/Oakland Seals',
    'OTT' : 'Ottawa Senators',
    'PHI' : 'Philadelphia Flyers',
    'PHX' : 'Phoenix Coyotes',
    'PIR' : 'Pittsburgh Pirates',
    'PIT' : 'Pittsburgh Penguins',
    'QBD' : 'Quebec Bulldogs',
    'QUA' : 'Philadelphia Quakers',
    'QUE' : 'Quebec Nordiques',
    'SEA' : 'Seattle Kraken',
    'SEN' : 'Ottawa Senators (original)',
    'SLE' : 'St. Louis Eagles',
    'SJS' : 'San Jose Sharks',
    'STL' : 'St. Louis Blues',
    'TAN' : 'Toronto Hockey Club/Toronto Arenas',
    'TBL' : 'Tampa Bay Lightning',
    'TOR' : 'Toronto Maple Leafs',
    'TSP' : 'Toronto St. Patricks',
    'VAN' : 'Vancouver Canucks',
    'VGK' : 'Vegas Golden Knights',
    'WIN' : 'Winnipeg Jets (original)',
    'WPG' : 'Winnipeg Jets',
    'WSH' : 'Washington Capitals'
}


# Append team features to player samples
player_df = player_dataset.alldata
team_df = team_dataset.alldata

samples_to_drop = []
for player in player_dataset.data:
    for N in player_dataset.data[player]:

        # For each group 
        for year in player_dataset.data[player][N]:

            # Find the team names for that group
            player_data = player_df[player_df['Player'] == player]
            player_data_year = player_data.loc[player_data['Season'].isin([season for season in range(year, year+N)])]
            team_names = player_data_year['Tm'].values

            # Get the team stats for each season
            drop = 0
            features = []
            for season,team in enumerate(team_names):

                if team not in team_name_dict:
                    # drop sample and clean up
                    drop = 1
                    samples_to_drop.append((player, N, year))
                    break

                team_name = team_name_dict[team]

                # Get normalized team data
                team_data = team_dataset.all_data_normalized[team_dataset.all_data_normalized['team name'] == team_name]
                team_data_year = team_data[team_data['Season'] == year+season]

                if team_data_year.empty:
                    # drop sample and clean up
                    drop = 1
                    samples_to_drop.append((player, N, year))
                    break
                
                features.append(torch.tensor(team_data_year.drop(columns=['team name', 'Season'],axis=1).values, dtype=torch.float32))

            if drop == 1:
                continue

            # Append the team stats to the player stats
            try:
                team_features = torch.stack(features).reshape((len(features), len(features[0][0]))) #features
            except:
                print(features)
                exit(0)
            player_features = player_dataset.data[player][N][year][0]
            new_features = torch.hstack((player_features, team_features))
            player_dataset.data[player][N][year] = (new_features , player_dataset.data[player][N][year])

# cleanup dataset
for (player, N, year) in samples_to_drop:
    player_dataset.data[player][N].pop(year, None)
    if len(player_dataset.data[player][N]) == 0: # remove N if empty
        player_dataset.data[player].pop(N, None)
        if len(player_dataset.data[player]) == 0: # remove player if empty
            player_dataset.data.pop(player, None)